In [5]:
# %pip install llama-index llama-index-vector-stores-upstash upstash-vector

  Using cached llama_index-0.11.1-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.3.0-py3-none-any.whl.metadata (728 bytes)
  Using cached llama_index_cli-0.3.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.11.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_openai-0.2.3-py3-none-any.whl.metadata (635 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.3.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_legacy-0.9.48.post3-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.2.0-py3-none-any.whl.metadata (648 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.2.0-py3-none-any.whl.metadata (728 bytes)
  Using cached llama_index_program_openai-0.2.0-py3-none-any.whl.metadata (766 bytes)
  Using cached llama_index_question_gen_openai-0.2.0-py3-none-any.whl.metadata (785 bytes)
  Using cached llama_index_readers_file-0.2.0-py3-none-any.whl.metadata (5.4 kB)

In [3]:
import setup
setup.init_django()

In [2]:
django_pdf = "data/django.pdf"

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore # changed from the LlamaIndex Docs

ModuleNotFoundError: No module named 'llama_index'

In [ ]:
reader = SimpleDirectoryReader(input_files=[django_pdf])

In [ ]:
import helpers
import openai
openai.api_key = helpers.config("OPENAI_API_KEY", default=None)

In [ ]:
url=helpers.config("UPSTASH_VECTOR_URL", default=None)
token=helpers.config("UPSTASH_VECTOR_TOKEN", default=None)
url

In [ ]:
data = reader.load_data()

In [ ]:
import httpx

timeout = httpx.Timeout(600.0, connect=600.0)
client = httpx.Client(timeout=timeout)

In [ ]:
from upstash_vector import Index

upstash_index = Index(
    url=url, 
    token=token,
    retries=5,
    retry_interval=0.2
)
upstash_index._client = client

In [ ]:
vector_store = UpstashVectorStore(
    url=url, 
    token=token,
)
vector_store._index = upstash_index

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(data, storage_context=storage_context, insert_batch_size=500)

In [ ]:
index_persist = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
from llama_index.core.response_synthesizers.type import ResponseMode
query_engine = index_persist.as_query_engine()

In [ ]:
r = query_engine.query("What are the arguments for CharField?")
print(r)